In [3]:
try:
    __file__
except:
    import findspark
    findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName("Sample") \
        .getOrCreate()

In [4]:
df = spark.read.format("com.crealytics.spark.excel").option("header", "true").load("airtravel.xlsx")

In [5]:
df.printSchema()

df.show()

root
 |-- Month: string (nullable = true)
 |-- year_1958: string (nullable = true)
 |-- year_1959: string (nullable = true)
 |-- year_1960: string (nullable = true)

+-----+---------+---------+---------+
|Month|year_1958|year_1959|year_1960|
+-----+---------+---------+---------+
|  JAN|      340|      360|      417|
|  FEB|      318|      342|      391|
|  MAR|      362|      406|      419|
|  APR|      348|      396|      461|
|  MAY|      363|      420|      472|
|  JUN|      435|      472|      535|
|  JUL|      491|      548|      622|
|  AUG|      505|      559|      606|
|  SEP|      404|      463|      508|
|  OCT|      359|      407|      461|
|  NOV|      310|      362|      390|
|  DEC|      337|      405|      432|
+-----+---------+---------+---------+



In [6]:
spark.stop()